In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import zipfile
 

%matplotlib inline

In [14]:
import os
os.chdir('/Users/Niloufar/Documents/Microsoft_Movie_Analysis')

In [112]:
#Load IMDB dataset
zf = zipfile.ZipFile('zippedData/im.db.zip', 'r')
conn = sqlite3.connect('im.db')
q = """
SELECT * FROM movie_basics
JOIN movie_ratings
    USING (movie_id);
"""
imdb = pd.read_sql(q, conn)

In [119]:
#Load BOM dataset
bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz')

In [114]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   original_title   73856 non-null  object 
 3   start_year       73856 non-null  int64  
 4   runtime_minutes  66236 non-null  float64
 5   genres           73052 non-null  object 
 6   averagerating    73856 non-null  float64
 7   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 4.5+ MB


In [115]:
bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [116]:
def prep_imdb(df):
    """This finction reads in the data from imdb and clean it"""
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df.drop(columns = ['movie_id', 'runtime_minutes','original_title'], inplace=True)
    df.rename({'primary_title':'title'}, axis=1, inplace=True)
    df['title'] = df['title'].str.title()
    df.rename({'start_year':'year'},axis=1, inplace=True)
    return df

imdb_df = prep_imdb(imdb)
imdb_df.head(3)

,title,year,genres,averagerating,numvotes
0,Sunghursh,2013,"Action,Crime,Drama",7.0,77
1,One Day Before The Rainy Season,2019,"Biography,Drama",7.2,43
2,The Other Side Of The Wind,2018,Drama,6.9,4517


In [121]:
def prep_bom(bom):
    """This function might read in and clean a different data source"""
    bom.columns = bom.columns.str.lower().str.replace(' ', '_')
    bom.drop(columns = ['studio'], inplace = True)
    bom['title'] = bom['title'].str.title()
    return bom

bom_df = prep_bom(bom)
bom_df.head(3)

,title,domestic_gross,foreign_gross,year
0,Toy Story 3,415000000.0,652000000,2010
1,Alice In Wonderland (2010),334200000.0,691300000,2010
2,Harry Potter And The Deathly Hallows Part 1,296000000.0,664300000,2010


In [151]:
df = pd.merge(imdb_df, bom_df, on=['title', 'year'])
df

,title,year,genres,averagerating,numvotes,domestic_gross,foreign_gross
0,Wazir,2016,"Action,Crime,Drama",7.1,15378,1100000.0,NaN
1,On The Road,2012,"Adventure,Drama,Romance",6.1,37886,744000.0,8000000
2,The Secret Life Of Walter Mitty,2013,"Adventure,Comedy,Drama",7.3,275300,58200000.0,129900000
3,A Walk Among The Tombstones,2014,"Action,Crime,Drama",6.5,105116,26300000.0,26900000
4,Jurassic World,2015,"Action,Adventure,Sci-Fi",7.0,539338,652300000.0,"1,019.4"
...,...,...,...,...,...,...,...
1916,The Spy Gone North,2018,Drama,7.2,1620,501000.0,NaN
1917,How Long Will I Love U,2018,Romance,6.5,607,747000.0,82100000
1918,Helicopter Eela,2018,Drama,5.4,673,72000.0,NaN
1919,Last Letter,2018,"Drama,Romance",6.4,322,181000.0,NaN


In [147]:
df.isna().sum()

title               0
year                0
genres              2
averagerating       0
numvotes            0
domestic_gross     11
foreign_gross     605
dtype: int64

In [169]:
df['foreign_gross'].str.replace(',', '').astype(float)

1         8000000.0
2       129900000.0
3        26900000.0
4            1019.4
5        10800000.0
           ...     
1904     35300000.0
1910     68700000.0
1912      1800000.0
1917     82100000.0
1920     16100000.0
Name: foreign_gross, Length: 1305, dtype: float64

In [171]:
df.dropna(subset= ['domestic_gross','foreign_gross'], inplace=True)
df['total_gross'] = df['domestic_gross'].astype(float) + df['foreign_gross'].str.replace(',', '').astype(float)

,title,year,genres,averagerating,numvotes,domestic_gross,foreign_gross,total_gross
1,On The Road,2012,"Adventure,Drama,Romance",6.1,37886,744000.0,8000000,8744000.0
2,The Secret Life Of Walter Mitty,2013,"Adventure,Comedy,Drama",7.3,275300,58200000.0,129900000,188100000.0
3,A Walk Among The Tombstones,2014,"Action,Crime,Drama",6.5,105116,26300000.0,26900000,53200000.0
4,Jurassic World,2015,"Action,Adventure,Sci-Fi",7.0,539338,652300000.0,"1,019.4",652301019.4
5,The Rum Diary,2011,"Comedy,Drama",6.2,94787,13100000.0,10800000,23900000.0
...,...,...,...,...,...,...,...,...
1904,Hereditary,2018,"Drama,Horror,Mystery",7.3,151571,44100000.0,35300000,79400000.0
1910,The Mule,2018,"Crime,Drama,Thriller",7.1,58955,103800000.0,68700000,172500000.0
1912,Nobody'S Fool,2018,"Comedy,Drama,Romance",4.6,3618,31700000.0,1800000,33500000.0
1917,How Long Will I Love U,2018,Romance,6.5,607,747000.0,82100000,82847000.0


In [175]:
df.sort_values(by=['total_gross'])

,title,year,genres,averagerating,numvotes,domestic_gross,foreign_gross,total_gross
512,Gasland,2010,Documentary,7.7,9940,30800.0,18600,4.940000e+04
208,Inhale,2010,"Drama,Thriller",6.6,6006,4100.0,51000,5.510000e+04
295,As Good As Dead,2010,"Crime,Thriller",5.2,1441,2000.0,53600,5.560000e+04
579,Band Baaja Baaraat,2010,"Comedy,Drama,Romance",7.2,13832,43800.0,60700,1.045000e+05
184,Freakonomics,2010,Documentary,6.4,6466,101000.0,16400,1.174000e+05
...,...,...,...,...,...,...,...,...
1044,Frozen,2013,"Adventure,Animation,Comedy",7.5,516998,400700000.0,875700000,1.276400e+09
1638,Jurassic World: Fallen Kingdom,2018,"Action,Adventure,Sci-Fi",6.2,219125,417700000.0,891800000,1.309500e+09
1167,Star Wars: The Last Jedi,2017,"Action,Adventure,Fantasy",7.1,462903,620200000.0,712400000,1.332600e+09
765,Black Panther,2018,"Action,Adventure,Sci-Fi",7.3,516148,700100000.0,646900000,1.347000e+09
